# Module 3: Create Shapefile from Classified Areas
Export classified areas as shapefiles.

Package installation requirements:
- Matplotlib (`matplotlib`): https://pypi.org/project/matplotlib/
- NumPy (`numpy`): https://pypi.org/project/numpy/
- Rasterio (`rasterio`): https://pypi.org/project/rasterio/
- Xarray (`xarray`): https://pypi.org/project/xarray/0.8.0rc1/
- Rasterio Xarray Extension (`rioxarray`): https://pypi.org/project/rioxarray/
- Shapely (`shapely`): https://pypi.org/project/shapely/
- Pandas (`pandas`): https://pypi.org/project/pandas/
- GeoPandas (`geopandas`): https://pypi.org/project/geopandas/
- Geocube (`geocube`): https://pypi.org/project/geocube/

***
## User Input
Define the key inputs for the module here:
1. Path to the dataset.
2. Data output path.

### Path to Data

In [ ]:
# define the path to the tiff output data from module 1 (or tiffs defined elsewhere)
try:
    req_id
except NameError:
    var_exists = False
    path_to_tiff = ''
else:
    var_exists = True

if var_exists:
    print('Found request ID generated from Module 1: '+req_id)
    path_to_tiff = f
else:
    if path_to_tiff == '':
        path_to_tiff = input('Please define the path to the GeoTIFF:')
    else:
        print('Path to GeoTIFF has been defined.')

## Output Path

In [ ]:
output_folder = ''
output_file = 'debris_outline.shp'

***
## Load Packages and Define Functions

In [ ]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import rioxarray
from shapely.geometry import shape
import geopandas as gpd
import pandas as pd
import shapely
from geocube.vector import vectorize

## Load Data

In [ ]:
# open tiff with rasterio
xda = rioxarray.open_rasterio(path_to_tiff)
print('GeoTIFF opened, '+str(len(xda.band))+' bands found.')

# define the spatial extent
lon = xda.x.data
lat = xda.y.data
extent = [np.min(lon), np.max(lon), np.max(lat), np.min(lat)]

xda

## Check Data

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
ax.set_title('Combined Classification of Floating Objects')
pos = ax.imshow(xda[0], cmap=plt.get_cmap('Greys', 2), extent=extent)
cbar = fig.colorbar(pos, ticks=[0,1], orientation='horizontal')
cbar.ax.set_xticklabels(['Not Debris', 'Debris'])  # horizontal colorbar
plt.ylabel('Latitude [degrees]')
plt.xlabel('Longitude [degrees]')
cbar.ax.set_xlabel('Classification')
plt.show()

## Create and Export Shapefile

In [ ]:
# read data and convert to the correct format
class_data = rioxarray.open_rasterio(path_to_tiff, mask_and_scale=True).squeeze()
class_data.name = "class"
class_data_i = class_data.astype(int)
class_gdf = vectorize(class_data_i)

In [ ]:
filtered_gdf = class_gdf[class_gdf['class'] == 1]
filtered_gdf

In [ ]:
# export shapefile
filtered_gdf.to_file(output_folder+output_file, driver='ESRI Shapefile') 